In [ ]:
!pip install pandas
!pip install binance-connector
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=61034763e98bc749a7b988884c4fb145bc1b1a1b495cea600f7c272903191967
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import logging
import pandas as pd
from binance.error import ClientError
from binance.error import ParameterArgumentError
from binance.error import ParameterRequiredError
from binance.error import ParameterTypeError
from binance.error import ParameterValueError
from binance.error import ServerError

logger = logging.getLogger(__name__)


class BinanceSpotMarketException(Exception):
    def __init__(
        self,
        err: Exception
        | ClientError
        | ServerError
        | ParameterRequiredError
        | ParameterValueError
        | ParameterTypeError
        | ParameterArgumentError,
    ):
        self.logger = logging.getLogger(__name__)
        self.error = err
        logging.error(self.error)

    def __str__(self):
        return f"BinanceSDK.Binance.Spot.Market.Exception: {self.error}"


In [ ]:
def singleton(class_):
    instances = {}

    def getinstance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]

    return getinstance


In [ ]:
import logging
from datetime import datetime
from typing import Any
from typing import Dict
from typing import List
from typing import NoReturn
from typing import Optional
from typing import Sequence
from typing import Union

from binance.spot import Spot
from pandas import DataFrame

import logging.config
import os


@singleton
class BinanceSpotMarket(object):
    def __init__(
        self, base_url: Optional[str] = "https://testnet.binance.vision"
    ) -> NoReturn:
        self.__apiKey = ""
        self.__apiSecret = ""
        self.__client = Spot(
            api_key=self.__apiKey, api_secret=self.__apiSecret, base_url=base_url
        )

    def kLines(
        self,
        symbol: str,
        interval: str,
        limit: Optional[int] = 10,
        startTime: Optional[int] = None,
        endTime: Optional[int] = None,
    ) -> Sequence[List["str"]]:
        try:
            output = self.__client.klines(
                symbol=symbol,
                interval=interval,
                limit=limit,
                startTime=startTime,
                endTime=endTime,
            )
            logging.debug(output)
            return output

        except Exception:
            raise BinanceSpotMarketException(err=Exception)

    @staticmethod
    def make_dataFrame(data):
        return DataFrame(data)

    def makeKLinesDataFrame(
        self,
        symbol: str,
        bar_interval,
        startTime: datetime,
        endTime: Optional[datetime],
    ) -> DataFrame:
          print(startTime)

          data = self.kLines(
              symbol=symbol,
              interval=bar_interval,
              limit=1000,
              startTime=startTime,
              endTime=endTime,
          )

          df = self.make_dataFrame(data)

          df.columns = [
              "Open time",
              "Open",
              "High",
              "Low",
              "Close",
              "Volume",
              "Kline Close time",
              "Quote asset volume",
              "Number of trades",
              "Taker buy base asset volume",
              "Taker buy quote asset volume",
              "Ignore",
          ]
          for i in list(df["Open time"]):
              df = df.replace(i, datetime.fromtimestamp(i / 1000))

          return df

    # def appendDataFrame(self, df1: DataFrame, ticker):
    #     df2 = self.makeKLinesDataFrame(
    #         symbol=ticker,
    #         bar_interval="1h",
    #         startTime=datetime(2022, 1, 1),
    #         endTime=datetime(2023, 3, 1),
    #     )
    #     # df1.loc[len(df2)] = list
    #     return df1


if __name__ == "__main__":
    b = BinanceSpotMarket()
    print(datetime.fromtimestamp(1643341200000 / 1000))
    df = b.makeKLinesDataFrame(
        symbol="BTCUSDT",
        bar_interval="1m",
        startTime=None,
        endTime=None,
    )

    print(df)


2022-01-28 03:40:00
None
              Open time            Open            High             Low  \
0   2023-05-24 07:44:00  26690.35000000  26702.81000000  26690.32000000   
1   2023-05-24 07:45:00  26702.81000000  26702.81000000  26700.43000000   
2   2023-05-24 07:46:00  26701.07000000  26702.05000000  26694.88000000   
3   2023-05-24 07:47:00  26694.88000000  26695.95000000  26694.88000000   
4   2023-05-24 07:48:00  26694.88000000  26700.49000000  26693.86000000   
..                  ...             ...             ...             ...   
995 2023-05-25 00:19:00  26314.87000000  26314.88000000  26301.13000000   
996 2023-05-25 00:20:00  26301.16000000  26301.17000000  26289.59000000   
997 2023-05-25 00:21:00  26289.32000000  26289.34000000  26276.55000000   
998 2023-05-25 00:22:00  26276.55000000  26284.49000000  26272.18000000   
999 2023-05-25 00:23:00  26277.94000000  26284.48000000  26274.18000000   

              Close      Volume  Kline Close time Quote asset volume  \
0 

In [ ]:
import logging
from datetime import datetime
from typing import Any, Dict, List, NoReturn, Optional, Sequence, Union
import csv
import pandas as pd
import pandas_ta as ta
from binance.spot import Spot
from pandas import DataFrame

logger = logging.getLogger(__name__)


class BinanceSpotMarketException(Exception):
    def __init__(self, err: Exception):
        self.logger = logging.getLogger(__name__)
        self.error = err
        logging.error(self.error)

    def __str__(self):
        return f"BinanceSDK.Binance.Spot.Market.Exception: {self.error}"


def singleton(class_):
    instances = {}

    def getinstance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]

    return getinstance


@singleton
class BinanceSpotMarket:
    def __init__(self, base_url: Optional[str] = "https://testnet.binance.vision") -> NoReturn:
        self.__apiKey = ""
        self.__apiSecret = ""
        self.__client = Spot(api_key=self.__apiKey, api_secret=self.__apiSecret, base_url=base_url)

    def kLines(
        self,
        symbol: str,
        interval: str,
        limit: Optional[int] = 10,
        startTime: Optional[int] = None,
        endTime: Optional[int] = None,
    ) -> Sequence[List[str]]:
        try:
            output = self.__client.klines(
                symbol=symbol,
                interval=interval,
                limit=limit,
                startTime=startTime,
                endTime=endTime,
            )
            logging.debug(output)
            return output

        except Exception:
            raise BinanceSpotMarketException(err=Exception)

    @staticmethod
    def make_dataFrame(data):
        return DataFrame(data)

    def makeKLinesDataFrame(
        self,
        symbol: str,
        bar_interval: str,
        startTime: Optional[datetime],
        endTime: Optional[datetime],
    ) -> DataFrame:
        print(startTime)

        data = self.kLines(
            symbol=symbol,
            interval=bar_interval,
            limit=1000,
            startTime=startTime,
            endTime=endTime,
        )

        df = self.make_dataFrame(data)

        df.columns = [
            "Open time",
            "Open",
            "High",
            "Low",
            "Close",
            "Volume",
            "Kline Close time",
            "Quote asset volume",
            "Number of trades",
            "Taker buy base asset volume",
            "Taker buy quote asset volume",
            "Ignore",
        ]
        for i in list(df["Open time"]):
            df = df.replace(i, datetime.fromtimestamp(i / 1000))

        return df


def calculate_rsi(df: pd.DataFrame, period: int = 14) -> pd.Series:
    """
    Рассчитывает RSI (Relative Strength Index) на основе pandas-ta.

    Аргументы:
        - df: pd.DataFrame - DataFrame с данными
        - period: int - период для расчета RSI (по умолчанию 14)

    Возвращает:
        - pd.Series - столбец со значениями RSI
    """
    df_rsi = ta.rsi(df["Close"], length=period)  # Расчет RSI с помощью pandas-ta

    # Обновление исходного DataFrame с новым столбцом "rsi"
    df["rsi"] = df_rsi
    print("df rsi", df["rsi"])
    return df["rsi"]


def backtest_smart_money_strategy():
    # Создание экземпляра BinanceSpotMarket
    market = BinanceSpotMarket()

    # Получение данных котировок
    symbol = "BTCUSDT"
    bar_interval = "1w"
    start_time = None
    end_time = None
    df = market.makeKLinesDataFrame(symbol, bar_interval, start_time, end_time)
    if start_time is None:
        start_time = datetime(2022, 1, 1)  # Примерная дата начала
    if end_time is None:
        end_time = datetime(2023, 3, 1)  # Примерная дата окончания

    print(df.tail())

    # Открытие файла для записи результатов
    with open("backtest_results.csv", mode="w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Time", "Action", "Balance USDT", "Balance Coin"])

        balance_usdt = 10000  # Начальный баланс в USDT
        balance_coin = 0  # Начальный баланс в Coin

        # Проходимся по каждой строке данных
        for index, row in df.iterrows():
            open_time = row["Open time"]
            close_time = row["Kline Close time"]
            price = row["Close"]
            rsi = calculate_rsi(df, period=14)  # Вычисляем значение RSI
            print(rsi)
            print(balance_usdt)
            print(balance_coin)

            # Проверяем условия для входа в позицию
            if not rsi.empty and rsi.iloc[-1] is not None and rsi.iloc[-1] > 70 and balance_usdt > 0:
                # Покупаем Coin
                balance_coin = balance_usdt / price
                balance_usdt = 0
                writer.writerow([close_time, "Buy", balance_usdt, balance_coin])

            # Проверяем условия для выхода из позиции
            if not rsi.empty and rsi.iloc[-1] is not None and rsi.iloc[-1] < 30 and balance_coin > 0:
                # Продаем Coin
                balance_usdt = balance_coin * price
                balance_coin = 0
                writer.writerow([close_time, "Sell", balance_usdt, balance_coin])

        # Записываем итоговый баланс
        writer.writerow(["Final Balance", "", balance_usdt, balance_coin])


if __name__ == "__main__":
    backtest_smart_money_strategy()

None
   Open time            Open            High             Low           Close  \
0 2023-05-01  28640.47000000  30350.92000000   6000.00000000  28428.42000000   
1 2023-05-08  28401.10000000  29856.63000000  10000.00000000  26917.56000000   
2 2023-05-15  26917.61000000  61000.00000000   9960.10000000  26747.78000000   
3 2023-05-22  26747.79000000  84807.47000000   5440.00000000  26306.67000000   

           Volume  Kline Close time  Quote asset volume  Number of trades  \
0  25169.42952400     1683503999999  729189061.16533280            729042   
1  33014.98375700     1684108799999  900025148.60042823           1078764   
2  29124.44351000     1684713599999  788065451.97562673            866519   
3  12784.13786800     1685318399999  343251459.66676364            413428   

  Taker buy base asset volume Taker buy quote asset volume Ignore  
0              14833.04813700           429755812.04788574      0  
1              19789.78341400           539558995.27910619      0  
2   